In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
from src.settings import MODELS_DIR

In [2]:
DEVICE = "cuda:0"

In [3]:
MODEL_NAME = "gpt2"
DATASET_NAME = "eli5"

In [4]:
from datasets import load_dataset

eli5 = load_dataset(DATASET_NAME, split="train_asks[:5000]")

In [5]:
eli5 = eli5.train_test_split(test_size=0.2)

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, device=DEVICE)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': 'c84vrt',
 'title': 'Air density affects your cardiovascular ability, but what about your strength?',
 'selftext': 'If I am wanting to lift a weight anywhere on the globe, what location on earth would be the best?',
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['eskh22i', 'esm63mu'],
 'answers.text': ["I'd say that your first rep is made without much of an oxygen impact since it would be fueled mostly by immediate atp reserves in muscles. Anyone correct me pls if I'm mistaken.\n\nOn a less serious note, it would be physically easier to lift (assuming you want to lift a typical gym dumbbell, much denser than air) in places of lesser air density and thus lesser air pressure; so I would look for a mountain ranges - although it won't make much of a difference that way.\n\nAnd if you had asked about lifting mass instead of weight, I guess the best place for it would be somewhere around the equator - that's possibly the furthest from the earth core you can be. (I'm 

In [8]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [9]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1106 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1291 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1092 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1121 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1095 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1437 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1819 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1382 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
OUT_MODEL_NAME = f"my_awesome_{DATASET_NAME}_{MODEL_NAME}"
MODEL_OUTPUT_DIR = MODELS_DIR / OUT_MODEL_NAME

In [15]:
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_DIR,
    evaluation_strategy="epoch",
    push_to_hub_model_id=OUT_MODEL_NAME,
    push_to_hub=True,
    hub_private_repo=True,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1716: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case jjamnicki/my_awesome_eli5_gpt2).
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.614700,3.512621


TrainOutput(global_step=1141, training_loss=3.646529415842737, metrics={'train_runtime': 315.817, 'train_samples_per_second': 28.903, 'train_steps_per_second': 3.613, 'total_flos': 596268417024000.0, 'train_loss': 3.646529415842737, 'epoch': 1.0})

In [16]:
PROMT = "Somatic hypermutation allows the immune system to"

In [17]:
from transformers import pipeline

generator = pipeline(
    task="text-generation",
    model=AutoModelForCausalLM.from_pretrained(OUT_MODEL_NAME, use_auth_token=True),
    tokenizer=AutoTokenizer.from_pretrained(OUT_MODEL_NAME, use_auth_token=True),
)
generator(PROMT)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Somatic hypermutation allows the immune system to avoid potentially dangerous viral infection from becoming too high.  Hypermutation has been studied extensively in this area and the effect is usually limited to a handful of different strains (HomoBacter, H'}]

In [18]:
inputs = tokenizer(PROMT, return_tensors="pt").input_ids
inputs

tensor([[   50, 13730,  8718,    76,  7094,  3578,   262, 10900,  1080,   284]])

In [19]:
outputs = model.generate(inputs.to(DEVICE), max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)
outputs

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[   50, 13730,  8718,    76,  7094,  3578,   262, 10900,  1080,   284,
          5529,   257,  8245,    11,  3487,  2858,    13,  6660,    11,   611,
           345,   460,   470,   651,  1576, 15568,   422,   262, 11492,   287,
           534,  2057,   329,   534,  1767,   284, 16274,    11,   262,  1767,
           460,   470,  2163,  6105,    13,   198,   198,    17,     8,   554,
          2846,   286, 25862,    11,   314,   460,   892,   286,   546,   257,
          3265,   286,   661,   351,   257,  7446, 15148,   422,   262,  2354,
            11,   508,   423, 19552,   257,  1178, 12796,   286,   262,  2641,
           326,   787,   606, 20807,  2092,   284,   262,  2354,    11,   981,
          1719, 19552,   281, 10238,  4866,   286,   262,  9779,   326,  5640,
           606,   284,   651,  6639,    13,  1002,   356,   821,  3375,   546]],
       device='cuda:0')

In [20]:
tokenizer.batch_decode(outputs, skip_special_tokens=True)

["Somatic hypermutation allows the immune system to maintain a stable, normal environment. Thus, if you can't get enough proteins from the bacteria in your food for your body to digest, the body can't function properly.\n\n2) In terms of genetics, I can think of about a population of people with a DNA mutation from the outside, who have inherited a few traits of the inside that make them genetically similar to the outside, while having inherited an underlying copy of the gene that causes them to get sick. If we're talking about"]